In [1]:
import pandas as pd
import numpy as np
import joblib

pd.options.display.max_columns = None
pd.options.display.max_rows = None

import warnings
warnings.filterwarnings("ignore")  

from expected_disposal_model.config import raw_file_path, modelling_file_path, preprocessor_file_path
from expected_disposal_model.data_preparation.data_preprocessor import Preprocessor
from expected_disposal_model.data_preparation.preprocessing import convert_chains_to_schema
from expected_disposal_model.data_preparation.preprocessing import filter_disposals, create_labels
from expected_disposal_model.data_preparation.preprocessing import get_stratified_train_test_val_columns
from expected_disposal_model.modelling_data_contract import ModellingDataContract


Load Data

In [2]:
data = pd.read_csv(raw_file_path)
data.head()

,Chain_Number,Initial_State,Final_State,Order,Quarter,Quarter_Duration_Chain_Start,Quarter_Duration,Team_Chain,Team,Player,AFL_API_Player_ID,Description,x,y,Disposal,Shot_At_Goal,Behind_Detail,Venue_Width,Venue_Length,Home_Team,Away_Team,Home_Team_Direction_Q1,Match_ID,Round_ID,Year,Season
0,1,centreBounce,goal,1.0,1,13,13.0,Brisbane Lions,NaN,NaN,NaN,Centre Bounce,0.0,0.0,NaN,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN
1,1,centreBounce,goal,2.0,1,13,24.0,Brisbane Lions,Brisbane Lions,Dayne Zorko,Dayne_Zorko,Hard Ball Get,8.0,-5.0,NaN,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN
2,1,centreBounce,goal,3.0,1,13,24.0,Brisbane Lions,Brisbane Lions,Dayne Zorko,Dayne_Zorko,Handball,9.0,-6.0,ineffective,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN
3,1,centreBounce,goal,4.0,1,13,28.0,Brisbane Lions,Sydney,Oliver Florent,Oliver_Florent,Loose Ball Get,11.0,-7.0,NaN,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN
4,1,centreBounce,goal,5.0,1,13,29.0,Brisbane Lions,Sydney,Oliver Florent,Oliver_Florent,Handball,12.0,-5.0,effective,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN


In [3]:
data.shape

(1287870, 26)

Preprocess Data

In [4]:
schema_chains = convert_chains_to_schema(data)
disposals = filter_disposals(schema_chains)

In [5]:
disposals.shape

(466216, 71)

In [6]:
preproc = Preprocessor()
preproc.fit(data)

Preprocessor()

In [7]:
X = preproc.transform(data)

In [8]:
X.head()

,Chain_Number,Initial_State,Final_State,Order,Quarter,Quarter_Duration_Chain_Start,Quarter_Duration,Team_Chain,Team,Player,AFL_API_Player_ID,Description,x,y,Disposal,Shot_At_Goal,Behind_Detail,Venue_Width,Venue_Length,Home_Team,Away_Team,Home_Team_Direction_Q1,Match_ID,Round_ID,Year,Season,action_type,Contested,Mark,Free,From_Centre_Bounce,Kick_Inside50,To_Ball_Up,From_Ball_Up,Rushed_Behind,Contest_Target,Goal,Behind,To_Out_On_Full,From_Out_On_Full,Error,From_Kick_In,end_x,end_y,pitch_start_x,pitch_start_y,pitch_end_x,pitch_end_y,left_right_start_x,left_right_start_y,left_right_end_x,left_right_end_y,start_distance_to_right_goal,end_distance_to_right_goal,Inside50,1,2,3,4,Duration,match_id,chain_number,order,quarter,quarter_seconds,overall_seconds,team,player,contested,mark,outcome_type,type_Kick_a0,type_Handball_a0,outcome_effective_a0,outcome_ineffective_a0,outcome_clanger_a0,type_Kick_outcome_effective_a0,type_Kick_outcome_ineffective_a0,type_Kick_outcome_clanger_a0,type_Handball_outcome_effective_a0,type_Handball_outcome_ineffective_a0,type_Handball_outcome_clanger_a0,quarter_a0,quarter_seconds_a0,overall_seconds_a0,left_right_start_x_a0,left_right_start_y_a0,left_right_end_x_a0,left_right_end_y_a0,dx_a0,dy_a0,movement_a0,contested_a0,mark_a0,type_Kick_a1,type_Handball_a1,outcome_effective_a1,outcome_ineffective_a1,outcome_clanger_a1,type_Kick_outcome_effective_a1,type_Kick_outcome_ineffective_a1,type_Kick_outcome_clanger_a1,type_Handball_outcome_effective_a1,type_Handball_outcome_ineffective_a1,type_Handball_outcome_clanger_a1,quarter_a1,quarter_seconds_a1,overall_seconds_a1,left_right_start_x_a1,left_right_start_y_a1,left_right_end_x_a1,left_right_end_y_a1,dx_a1,dy_a1,movement_a1,contested_a1,mark_a1,type_Kick_a2,type_Handball_a2,outcome_effective_a2,outcome_ineffective_a2,outcome_clanger_a2,type_Kick_outcome_effective_a2,type_Kick_outcome_ineffective_a2,type_Kick_outcome_clanger_a2,type_Handball_outcome_effective_a2,type_Handball_outcome_ineffective_a2,type_Handball_outcome_clanger_a2,quarter_a2,quarter_seconds_a2,overall_seconds_a2,left_right_start_x_a2,left_right_start_y_a2,left_right_end_x_a2,left_right_end_y_a2,dx_a2,dy_a2,movement_a2,contested_a2,mark_a2,team_1,team_2,time_delta1,time_delta2,dx_a01,dy_a01,move_a01,dx_a02,dy_a02,move_a02,goalscore_team,goalscore_opponent,goalscore_diff
0,1,centreBounce,goal,3.0,1,13,24.0,Brisbane Lions,Brisbane Lions,Dayne Zorko,Dayne_Zorko,Handball,9.0,-6.0,ineffective,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN,Handball,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,-5.0,9.0,-6.0,12.0,-5.0,9.0,-6.0,12.0,-5.0,69.260378,66.189123,NaN,1689.0,2119.0,2063.0,1920.0,24.0,202101_BrisbaneLions_Sydney,1,3.0,1,24.0,24.0,Brisbane Lions,Dayne Zorko,NaN,NaN,ineffective,False,True,False,True,False,False,False,False,False,True,False,1,24.0,24.0,9.0,-6.0,12.0,-5.0,3.0,1.0,3.162278,NaN,NaN,False,True,False,True,False,False,False,False,False,True,False,1,24.0,24.0,9.0,-6.0,12.0,-5.0,3.0,1.0,3.162278,NaN,NaN,False,True,False,True,False,False,False,False,False,True,False,1,24.0,24.0,9.0,-6.0,12.0,-5.0,3.0,1.0,3.162278,NaN,NaN,True,True,0.0,0.0,3.0,1.0,3.162278,3.0,1.0,3.162278,0,0,0
1,1,centreBounce,goal,5.0,1,13,29.0,Brisbane Lions,Sydney,Oliver Florent,Oliver_Florent,Handball,12.0,-5.0,effective,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN,Handball,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.0,-2.0,12.0,-5.0,22.0,-2.0,-12.0,5.0,-22.0,2.0,90.138782,100.019998,NaN,1689.0,2119.0,2063.0,1920.0,29.0,202101_BrisbaneLions_Sydney,1,5.0,1,29.0,29.0,Sydney,Oliver Florent,NaN,NaN,effective,False,True,True,False,False,False,False,False,True,False,False,1,29.0,29.0,-12.0,5.0,-22.0,2.0,-10.0,-3.0,10.440307,NaN,NaN,False,True,False,True,False,False,False,False,False,True,False,1,24.0,24.0,9.0,-6.0,12.0,-5.0,3.0,1.0,3.162278,NaN,NaN,False,True,False,True,False,False,False,False,False,T

In [9]:
X.shape

(466216, 153)

Create Labels

In [10]:
y = create_labels(data)

In [11]:
y.shape

(466216,)

In [12]:
y.head()

0    0
1    0
2    1
3    0
4    1
Name: Disposal_Response, dtype: int64

Combine Data

In [13]:
modelling_data = pd.concat([X, y], axis='columns')
modelling_data.head()

,Chain_Number,Initial_State,Final_State,Order,Quarter,Quarter_Duration_Chain_Start,Quarter_Duration,Team_Chain,Team,Player,AFL_API_Player_ID,Description,x,y,Disposal,Shot_At_Goal,Behind_Detail,Venue_Width,Venue_Length,Home_Team,Away_Team,Home_Team_Direction_Q1,Match_ID,Round_ID,Year,Season,action_type,Contested,Mark,Free,From_Centre_Bounce,Kick_Inside50,To_Ball_Up,From_Ball_Up,Rushed_Behind,Contest_Target,Goal,Behind,To_Out_On_Full,From_Out_On_Full,Error,From_Kick_In,end_x,end_y,pitch_start_x,pitch_start_y,pitch_end_x,pitch_end_y,left_right_start_x,left_right_start_y,left_right_end_x,left_right_end_y,start_distance_to_right_goal,end_distance_to_right_goal,Inside50,1,2,3,4,Duration,match_id,chain_number,order,quarter,quarter_seconds,overall_seconds,team,player,contested,mark,outcome_type,type_Kick_a0,type_Handball_a0,outcome_effective_a0,outcome_ineffective_a0,outcome_clanger_a0,type_Kick_outcome_effective_a0,type_Kick_outcome_ineffective_a0,type_Kick_outcome_clanger_a0,type_Handball_outcome_effective_a0,type_Handball_outcome_ineffective_a0,type_Handball_outcome_clanger_a0,quarter_a0,quarter_seconds_a0,overall_seconds_a0,left_right_start_x_a0,left_right_start_y_a0,left_right_end_x_a0,left_right_end_y_a0,dx_a0,dy_a0,movement_a0,contested_a0,mark_a0,type_Kick_a1,type_Handball_a1,outcome_effective_a1,outcome_ineffective_a1,outcome_clanger_a1,type_Kick_outcome_effective_a1,type_Kick_outcome_ineffective_a1,type_Kick_outcome_clanger_a1,type_Handball_outcome_effective_a1,type_Handball_outcome_ineffective_a1,type_Handball_outcome_clanger_a1,quarter_a1,quarter_seconds_a1,overall_seconds_a1,left_right_start_x_a1,left_right_start_y_a1,left_right_end_x_a1,left_right_end_y_a1,dx_a1,dy_a1,movement_a1,contested_a1,mark_a1,type_Kick_a2,type_Handball_a2,outcome_effective_a2,outcome_ineffective_a2,outcome_clanger_a2,type_Kick_outcome_effective_a2,type_Kick_outcome_ineffective_a2,type_Kick_outcome_clanger_a2,type_Handball_outcome_effective_a2,type_Handball_outcome_ineffective_a2,type_Handball_outcome_clanger_a2,quarter_a2,quarter_seconds_a2,overall_seconds_a2,left_right_start_x_a2,left_right_start_y_a2,left_right_end_x_a2,left_right_end_y_a2,dx_a2,dy_a2,movement_a2,contested_a2,mark_a2,team_1,team_2,time_delta1,time_delta2,dx_a01,dy_a01,move_a01,dx_a02,dy_a02,move_a02,goalscore_team,goalscore_opponent,goalscore_diff,Disposal_Response
0,1,centreBounce,goal,3.0,1,13,24.0,Brisbane Lions,Brisbane Lions,Dayne Zorko,Dayne_Zorko,Handball,9.0,-6.0,ineffective,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN,Handball,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,-5.0,9.0,-6.0,12.0,-5.0,9.0,-6.0,12.0,-5.0,69.260378,66.189123,NaN,1689.0,2119.0,2063.0,1920.0,24.0,202101_BrisbaneLions_Sydney,1,3.0,1,24.0,24.0,Brisbane Lions,Dayne Zorko,NaN,NaN,ineffective,False,True,False,True,False,False,False,False,False,True,False,1,24.0,24.0,9.0,-6.0,12.0,-5.0,3.0,1.0,3.162278,NaN,NaN,False,True,False,True,False,False,False,False,False,True,False,1,24.0,24.0,9.0,-6.0,12.0,-5.0,3.0,1.0,3.162278,NaN,NaN,False,True,False,True,False,False,False,False,False,True,False,1,24.0,24.0,9.0,-6.0,12.0,-5.0,3.0,1.0,3.162278,NaN,NaN,True,True,0.0,0.0,3.0,1.0,3.162278,3.0,1.0,3.162278,0,0,0,0
1,1,centreBounce,goal,5.0,1,13,29.0,Brisbane Lions,Sydney,Oliver Florent,Oliver_Florent,Handball,12.0,-5.0,effective,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN,Handball,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.0,-2.0,12.0,-5.0,22.0,-2.0,-12.0,5.0,-22.0,2.0,90.138782,100.019998,NaN,1689.0,2119.0,2063.0,1920.0,29.0,202101_BrisbaneLions_Sydney,1,5.0,1,29.0,29.0,Sydney,Oliver Florent,NaN,NaN,effective,False,True,True,False,False,False,False,False,True,False,False,1,29.0,29.0,-12.0,5.0,-22.0,2.0,-10.0,-3.0,10.440307,NaN,NaN,False,True,False,True,False,False,False,False,False,True,False,1,24.0,24.0,9.0,-6.0,12.0,-5.0,3.0,1.0,3.162278,NaN,NaN,False,True,False,True,False,False

In [14]:
modelling_data.shape

(466216, 154)

In [15]:
del X
del y
del disposals
del data
del schema_chains

Create Train Test Validation Split

In [16]:
modelling_data = get_stratified_train_test_val_columns(modelling_data, response=ModellingDataContract.RESPONSE)
modelling_data.head()

,Chain_Number,Initial_State,Final_State,Order,Quarter,Quarter_Duration_Chain_Start,Quarter_Duration,Team_Chain,Team,Player,AFL_API_Player_ID,Description,x,y,Disposal,Shot_At_Goal,Behind_Detail,Venue_Width,Venue_Length,Home_Team,Away_Team,Home_Team_Direction_Q1,Match_ID,Round_ID,Year,Season,action_type,Contested,Mark,Free,From_Centre_Bounce,Kick_Inside50,To_Ball_Up,From_Ball_Up,Rushed_Behind,Contest_Target,Goal,Behind,To_Out_On_Full,From_Out_On_Full,Error,From_Kick_In,end_x,end_y,pitch_start_x,pitch_start_y,pitch_end_x,pitch_end_y,left_right_start_x,left_right_start_y,left_right_end_x,left_right_end_y,start_distance_to_right_goal,end_distance_to_right_goal,Inside50,1,2,3,4,Duration,match_id,chain_number,order,quarter,quarter_seconds,overall_seconds,team,player,contested,mark,outcome_type,type_Kick_a0,type_Handball_a0,outcome_effective_a0,outcome_ineffective_a0,outcome_clanger_a0,type_Kick_outcome_effective_a0,type_Kick_outcome_ineffective_a0,type_Kick_outcome_clanger_a0,type_Handball_outcome_effective_a0,type_Handball_outcome_ineffective_a0,type_Handball_outcome_clanger_a0,quarter_a0,quarter_seconds_a0,overall_seconds_a0,left_right_start_x_a0,left_right_start_y_a0,left_right_end_x_a0,left_right_end_y_a0,dx_a0,dy_a0,movement_a0,contested_a0,mark_a0,type_Kick_a1,type_Handball_a1,outcome_effective_a1,outcome_ineffective_a1,outcome_clanger_a1,type_Kick_outcome_effective_a1,type_Kick_outcome_ineffective_a1,type_Kick_outcome_clanger_a1,type_Handball_outcome_effective_a1,type_Handball_outcome_ineffective_a1,type_Handball_outcome_clanger_a1,quarter_a1,quarter_seconds_a1,overall_seconds_a1,left_right_start_x_a1,left_right_start_y_a1,left_right_end_x_a1,left_right_end_y_a1,dx_a1,dy_a1,movement_a1,contested_a1,mark_a1,type_Kick_a2,type_Handball_a2,outcome_effective_a2,outcome_ineffective_a2,outcome_clanger_a2,type_Kick_outcome_effective_a2,type_Kick_outcome_ineffective_a2,type_Kick_outcome_clanger_a2,type_Handball_outcome_effective_a2,type_Handball_outcome_ineffective_a2,type_Handball_outcome_clanger_a2,quarter_a2,quarter_seconds_a2,overall_seconds_a2,left_right_start_x_a2,left_right_start_y_a2,left_right_end_x_a2,left_right_end_y_a2,dx_a2,dy_a2,movement_a2,contested_a2,mark_a2,team_1,team_2,time_delta1,time_delta2,dx_a01,dy_a01,move_a01,dx_a02,dy_a02,move_a02,goalscore_team,goalscore_opponent,goalscore_diff,Disposal_Response,Disposal_ResponseTrainingSet,Disposal_ResponseTestSet,Disposal_ResponseValidationSet
0,1,centreBounce,goal,3.0,1,13,24.0,Brisbane Lions,Brisbane Lions,Dayne Zorko,Dayne_Zorko,Handball,9.0,-6.0,ineffective,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN,Handball,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,-5.0,9.0,-6.0,12.0,-5.0,9.0,-6.0,12.0,-5.0,69.260378,66.189123,NaN,1689.0,2119.0,2063.0,1920.0,24.0,202101_BrisbaneLions_Sydney,1,3.0,1,24.0,24.0,Brisbane Lions,Dayne Zorko,NaN,NaN,ineffective,False,True,False,True,False,False,False,False,False,True,False,1,24.0,24.0,9.0,-6.0,12.0,-5.0,3.0,1.0,3.162278,NaN,NaN,False,True,False,True,False,False,False,False,False,True,False,1,24.0,24.0,9.0,-6.0,12.0,-5.0,3.0,1.0,3.162278,NaN,NaN,False,True,False,True,False,False,False,False,False,True,False,1,24.0,24.0,9.0,-6.0,12.0,-5.0,3.0,1.0,3.162278,NaN,NaN,True,True,0.0,0.0,3.0,1.0,3.162278,3.0,1.0,3.162278,0,0,0,0,False,False,True
1,1,centreBounce,goal,5.0,1,13,29.0,Brisbane Lions,Sydney,Oliver Florent,Oliver_Florent,Handball,12.0,-5.0,effective,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN,Handball,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.0,-2.0,12.0,-5.0,22.0,-2.0,-12.0,5.0,-22.0,2.0,90.138782,100.019998,NaN,1689.0,2119.0,2063.0,1920.0,29.0,202101_BrisbaneLions_Sydney,1,5.0,1,29.0,29.0,Sydney,Oliver Florent,NaN,NaN,effective,False,True,True,False,False,False,False,False,True,False,False,1,29.0,29.0,-12.0,5.0,-22.0,2.0,-10.0,-3.0,10.440307,NaN,NaN,False,True,False,True,False,False,False,False,Fals

Export Data

In [17]:
modelling_file_path

'/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-disposal-model/data/disposal_modelling_data_v2.csv'

In [18]:
modelling_data.to_csv(modelling_file_path, index=False)

Export Preprocessor

In [19]:
preprocessor_file_path = '/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-disposal-model/model_outputs/preprocessors/disposal_preprocessor_v2.joblib'

In [20]:
preprocessor_file_path

'/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-disposal-model/model_outputs/preprocessors/disposal_preprocessor_v2.joblib'

In [21]:
joblib.dump(preproc, preprocessor_file_path)

['/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-disposal-model/model_outputs/preprocessors/disposal_preprocessor_v2.joblib']